In [1]:
# Basic
from collections import Counter
import datetime
import inspect
import math
import numpy as np
import os
import pprint
import sys

# yaml specific
import yaml

# Data handling
from fuzzywuzzy import fuzz, process
import pandas as pd
from tqdm import tqdm

# This is to add the root project folder to sys.path so that imports are easy
currentdir = os.path.dirname(os.path.abspath(inspect.getfile(inspect.currentframe())))
parentdir = os.path.dirname(currentdir)
sys.path.insert(0,parentdir) 

# my library
from src.db_utils import update_player, add_player 
from src.core import runs_scored
from src.insights import batting_total_runs

C:\Users\vivsu\anaconda3\envs\cricboard\lib\site-packages\fuzzywuzzy\fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')
C:\Users\vivsu\anaconda3\envs\cricboard\lib\site-packages\IPython\core\interactiveshell.py:3357: DtypeWarning: Columns (3) have mixed types.Specify dtype option on import or set low_memory=False.
  if (await self.run_code(code, result,  async_=asy)):


In [2]:
# Config variables
raw_data_path = os.path.join("..", "raw_data") 
clean_data_path = os.path.join("..", "clean_data") 

In [3]:
df_tournament = pd.read_csv(os.path.join(clean_data_path, "tournament.csv"))
df_tournament = df_tournament.loc[:, ~df_tournament.columns.str.contains('^Unnamed')]
tournament_id_map = dict(zip(df_tournament.tournament_name, df_tournament.tournament_id))

In [4]:
df_venue = pd.read_csv(os.path.join(clean_data_path, "venue.csv"))
df_venue = df_venue.loc[:, ~df_venue.columns.str.contains('^Unnamed')]
venue_id_map = dict(zip(df_venue.venue_name, df_venue.venue_id))

# Mapping the duplicates as well to its correct venue ids
# IPL maps
venue_id_map["Punjab Cricket Association IS Bindra Stadium, Mohali"] = venue_id_map["Punjab Cricket Association Stadium, Mohali"]
venue_id_map["M.Chinnaswamy Stadium"] = venue_id_map["M Chinnaswamy Stadium"]
# BBL maps
#venue_id_map["Brisbane Cricket Ground"] = venue_id_map["Brisbane Cricket Ground, Woolloongabba"]

In [5]:
df_team = pd.read_csv(os.path.join(clean_data_path, "team.csv"))
df_team = df_team.loc[:, ~df_team.columns.str.contains('^Unnamed')]
team_id_map = dict(zip(df_team.team_name, df_team.team_id))

In [6]:
df_player = pd.read_csv(os.path.join(clean_data_path, "player.csv"))
df_player = df_player.loc[:, ~df_player.columns.str.contains('^Unnamed')]
player_dispname_id_map = dict(zip(df_player.player_display_name, df_player.player_id))
player_name_id_map = dict(zip(df_player.player_name, df_player.player_id))

In [7]:
df_match = pd.read_csv(os.path.join(clean_data_path, "match.csv"))
df_match = df_match.loc[:, ~df_match.columns.str.contains('^Unnamed')]

df_ball = pd.read_csv(os.path.join(clean_data_path, "ball.csv"))
df_ball = df_ball.loc[:, ~df_ball.columns.str.contains('^Unnamed')]

C:\Users\vivsu\anaconda3\envs\cricboard\lib\site-packages\IPython\core\interactiveshell.py:3165: DtypeWarning: Columns (3) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [20]:
player_name = "Virat Kohli"
player_id = player_name_id_map[player_name]
venue_name = "ALL"
match_format = "T20"
tournaments = None
years_range = [str(year) for year in range(2000, 2022)]
overs_range = [0, 20]
top_n=20

In [21]:
balls_by_pace=runs_scored(player=player_id, tournaments=tournaments, venue=None, years=None, overs_range=None, against_spin=None, against_pace=True, against_bowler=None)
balls_by_pace

,ball_id,match_id,ball_number,innings_number,batsman,bowler,non_striker,batsman_runs,wide_runs,bye_runs,noball_runs,legbye_runs,extras_runs,total_runs,player_dismissed,dismissal_type,fielders,ball_description,ball_url
2590,2590,11,0.2,1,2422,1997,9246,0,1,0,0,0,1,1,NaN,NaN,NaN,NaN,NaN
2591,2591,11,0.3,1,2422,1997,9246,1,0,0,0,0,0,1,NaN,NaN,NaN,NaN,NaN
2593,2593,11,0.5,1,2422,1997,9246,0,0,0,0,0,0,0,NaN,NaN,NaN,NaN,NaN
2594,2594,11,0.6,1,2422,1997,9246,1,0,0,0,0,0,1,NaN,NaN,NaN,NaN,NaN
2603,2603,11,2.1,1,2422,1997,9246,6,0,0,0,0,0,6,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
193562,193562,815,10.6,2,2422,7256,8304,0,0,0,0,0,0,0,NaN,NaN,NaN,NaN,NaN
193570,193570,815,12.2,2,2422,2659,8304,6,0,0,0,0,0,6,NaN,NaN,NaN,NaN,NaN
193571,193571,815,12.3,2,2422,2659,8304,2,0,0,0,0,0,2,NaN,NaN,NaN,NaN,NaN
193572,193572,815,12.4,2,2422,2659,8304,2,0,0,0,0,0,2,NaN,NaN,NaN,NaN,NaN


In [22]:
balls_by_spin=runs_scored(player=player_id, tournaments=tournaments, venue=None, years=None, overs_range=None, against_spin=True, against_pace=None, against_bowler=None)

In [23]:
balls_by_spin

,ball_id,match_id,ball_number,innings_number,batsman,bowler,non_striker,batsman_runs,wide_runs,bye_runs,noball_runs,legbye_runs,extras_runs,total_runs,player_dismissed,dismissal_type,fielders,ball_description,ball_url
2597,2597,11,1.1,1,2422,2391,9246,0,0,0,0,0,0,0,NaN,NaN,NaN,NaN,NaN
2598,2598,11,1.2,1,2422,2391,9246,1,0,0,0,0,0,1,NaN,NaN,NaN,NaN,NaN
2620,2620,11,4.6,1,2422,2391,9246,1,0,0,0,0,0,1,NaN,NaN,NaN,NaN,NaN
2627,2627,11,6.1,1,2422,2391,9246,1,0,0,0,0,0,1,NaN,NaN,NaN,NaN,NaN
2639,2639,11,8.1,1,2422,2391,9246,4,0,0,0,0,0,4,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
192619,192619,811,14.5,2,2422,2604,8034,1,0,0,0,0,0,1,NaN,NaN,NaN,NaN,NaN
192622,192622,811,15.2,2,2422,2545,2161,1,0,0,0,0,0,1,NaN,NaN,NaN,NaN,NaN
193564,193564,815,11.2,2,2422,2160,8304,4,0,0,0,0,0,4,NaN,NaN,NaN,NaN,NaN
193565,193565,815,11.3,2,2422,2160,8304,1,0,0,0,0,0,1,NaN,NaN,NaN,NaN,NaN


In [24]:
all_balls=runs_scored(player=player_id, tournaments=tournaments, venue=None, years=None, overs_range=None, against_spin=None, against_pace=None, against_bowler=None)

In [25]:
all_balls

,ball_id,match_id,ball_number,innings_number,batsman,bowler,non_striker,batsman_runs,wide_runs,bye_runs,noball_runs,legbye_runs,extras_runs,total_runs,player_dismissed,dismissal_type,fielders,ball_description,ball_url
2590,2590,11,0.2,1,2422,1997,9246,0,1,0,0,0,1,1,NaN,NaN,NaN,NaN,NaN
2591,2591,11,0.3,1,2422,1997,9246,1,0,0,0,0,0,1,NaN,NaN,NaN,NaN,NaN
2593,2593,11,0.5,1,2422,1997,9246,0,0,0,0,0,0,0,NaN,NaN,NaN,NaN,NaN
2594,2594,11,0.6,1,2422,1997,9246,1,0,0,0,0,0,1,NaN,NaN,NaN,NaN,NaN
2597,2597,11,1.1,1,2422,2391,9246,0,0,0,0,0,0,0,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
193567,193567,815,11.5,2,2422,2160,8304,1,0,0,0,0,0,1,NaN,NaN,NaN,NaN,NaN
193570,193570,815,12.2,2,2422,2659,8304,6,0,0,0,0,0,6,NaN,NaN,NaN,NaN,NaN
193571,193571,815,12.3,2,2422,2659,8304,2,0,0,0,0,0,2,NaN,NaN,NaN,NaN,NaN
193572,193572,815,12.4,2,2422,2659,8304,2,0,0,0,0,0,2,NaN,NaN,NaN,NaN,NaN


In [26]:
a=all_balls[~all_balls['ball_id'].isin(np.array(balls_by_pace['ball_id']))]

In [27]:
b=a[~a['ball_id'].isin(np.array(balls_by_spin['ball_id']))]

In [28]:
b

,ball_id,match_id,ball_number,innings_number,batsman,bowler,non_striker,batsman_runs,wide_runs,bye_runs,noball_runs,legbye_runs,extras_runs,total_runs,player_dismissed,dismissal_type,fielders,ball_description,ball_url
4622,4622,19,10.2,1,2422,7917,9246,1,0,0,0,0,0,1,NaN,NaN,NaN,NaN,NaN
4627,4627,19,10.7,1,2422,7917,9246,1,0,0,0,0,0,1,NaN,NaN,NaN,NaN,NaN
189784,189784,799,9.1,1,2422,7917,8304,2,0,0,0,0,0,2,NaN,NaN,NaN,NaN,NaN
189785,189785,799,9.2,1,2422,7917,8304,4,0,0,0,0,0,4,NaN,NaN,NaN,NaN,NaN
189786,189786,799,9.3,1,2422,7917,8304,1,0,0,0,0,0,1,NaN,NaN,NaN,NaN,NaN
189788,189788,799,9.5,1,2422,7917,8304,1,0,0,0,0,0,1,NaN,NaN,NaN,NaN,NaN


In [29]:
set(np.array(b['bowler']))

{7917}

In [30]:
set(np.array(b['match_id']))

{19, 799}

In [31]:
balls_by_pace['batsman_runs'].sum() + balls_by_spin['batsman_runs'].sum()

5878